In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
usuario = "root"
senha = "root"
url = "localhost"
porta = "3306"
banco = "ENEM_DM"

# Criar uma conexão com o banco de dados MySQL
engine_dm = create_engine(f"mysql+pymysql://{usuario}:{senha}@{url}:{porta}/{banco}")

In [16]:
query = """
  WITH CTE AS (
      SELECT
        COR_RACA,
        SEXO,
        AVG(
          NU_NOTA_REDACAO
        ) AS MEDIA_REDACAO
      FROM
        FAT_NOTAS
        JOIN DIM_PARTICIPANTES USING (SK_PARTICIPANTES)
        JOIN DIM_SEXO USING (SK_SEXO)
        JOIN DIM_COR_RACA USING (SK_COR_RACA)
        JOIN DIM_STATUS_REDACAO USING (SK_STATUS_REDACAO)
    WHERE
      TP_STATUS_REDACAO = 1
    GROUP BY
      COR_RACA,
      SEXO
  )
  SELECT
    *,
    RANK() OVER (
      ORDER BY
        MEDIA_REDACAO DESC
    ) AS RNK
  FROM
    CTE
  ORDER BY
    RNK
"""

df = pd.read_sql_query(query, con=engine_dm)

display(df)

,COR_RACA,SEXO,MEDIA_REDACAO,RNK
0,Branca,Feminino,709.38498,1
1,Branca,Masculino,672.62476,2
2,Não declarado,Feminino,655.93132,3
3,Parda,Feminino,643.89233,4
4,Amarela,Feminino,636.62697,5
5,Amarela,Masculino,629.34235,6
6,Preta,Feminino,628.16692,7
7,Parda,Masculino,619.97137,8
8,Não declarado,Masculino,617.61613,9
9,Preta,Masculino,603.67247,10


In [11]:
query = """
  WITH CTE AS (
      SELECT
        COR_RACA,
        SEXO,
        SUM(
          NU_NOTA_CN + NU_NOTA_CH + NU_NOTA_LC + NU_NOTA_MT
        ) / 4 AS MEDIA_PROVA
      FROM
        FAT_NOTAS
        JOIN DIM_PARTICIPANTES USING (SK_PARTICIPANTES)
        JOIN DIM_SEXO USING (SK_SEXO)
        JOIN DIM_COR_RACA USING (SK_COR_RACA)
      GROUP BY
        COR_RACA,
        SEXO
  )
  SELECT
    *,
    RANK() OVER (
      ORDER BY
        MEDIA_PROVA DESC
    ) AS RNK
  FROM
    CTE
  ORDER BY
    RNK
"""

df = pd.read_sql_query(query, con=engine_dm)

display(df)

,COR_RACA,SEXO,MEDIA_PROVA,RNK
0,Branca,Feminino,1.086325e+08,1
1,Parda,Feminino,1.075389e+08,2
2,Parda,Masculino,5.353353e+07,3
3,Branca,Masculino,5.134051e+07,4
4,Preta,Feminino,3.103968e+07,5
5,Preta,Masculino,1.637159e+07,6
6,Amarela,Feminino,4.100959e+06,7
7,Não declarado,Feminino,2.915902e+06,8
8,Amarela,Masculino,1.725156e+06,9
9,Não declarado,Masculino,1.511070e+06,10


In [15]:
query = """
  SELECT
    COR_RACA,
    SEXO,
    ESTADO_CIVIL,
    COALESCE(AVG(NU_NOTA_MT), 0) AS MEDIA_MT,
    RANK() OVER (
      ORDER BY
        COALESCE(AVG(NU_NOTA_MT), 0) DESC
    ) AS RNK
  FROM
    FAT_NOTAS
    JOIN DIM_PARTICIPANTES USING (SK_PARTICIPANTES)
    JOIN DIM_SEXO USING (SK_SEXO)
    JOIN DIM_COR_RACA USING (SK_COR_RACA)
    JOIN DIM_ESTADO_CIVIL USING (SK_ESTADO_CIVIL)
    JOIN DIM_PRESENCA_MT USING (SK_PRESENCA_MT)
  WHERE
    TP_PRESENCA_MT = 1
  GROUP BY
    COR_RACA,
    SEXO,
    ESTADO_CIVIL
  ORDER BY
    RNK;
"""

df = pd.read_sql_query(query, con=engine_dm)

display(df)

,COR_RACA,SEXO,ESTADO_CIVIL,MEDIA_MT,RNK
0,Não declarado,Masculino,Viúvo(a),654.50000,1
1,Branca,Masculino,Solteiro(a),598.10479,2
2,Branca,Masculino,Não informado,576.26460,3
3,Não declarado,Masculino,Casado(a)/Mora com companheiro(a),570.72821,4
4,Branca,Masculino,Divorciado(a)/Desquitado(a)/Separado(a),569.94752,5
5,Amarela,Masculino,Solteiro(a),565.92747,6
6,Branca,Masculino,Casado(a)/Mora com companheiro(a),565.29842,7
7,Branca,Feminino,Solteiro(a),553.81399,8
8,Não declarado,Masculino,Solteiro(a),550.46665,9
9,Amarela,Masculino,Divorciado(a)/Desquitado(a)/Separado(a),545.71538,10


In [17]:
query = """
  with
aux as (
    select
        p.NU_INSCRICAO,
        p.NU.NOTA.REDACAO,
        p.TP.SEXO,
        q.Q005,
        row_number() over (partition by p.NU_INSCRICAO order by n.NU_NOTA_REDACAO desc) as rn
    from
        participantes p
    left join
        notas n on n.NU_INSCRICAO = p.NU_INSCRICAO
    left join
         questionario q on q.NU_INSCRICAO = p. NU_INSCRICAO
    where
        TP_ST_CONCLUSAO = 2
        and TP_COR_RACA = 4
        and TP_FAIXA_ETARIA = 2
)
select
    p.NU_INSCRICAO,
    p.NU.NOTA.REDACAO,
    p.TP.SEXO,
    avg(q.Q005) as media_pessoas_residencia
from
    aux
where
    rn = 1
group by 1, 2, 3
"""

df = pd.read_sql_query(query, con=engine_dm)

display(df)

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '.REDACAO,\n        p.TP.SEXO,\n        q.Q005,\n        row_number() over (partitio' at line 5")
[SQL: 
  with 
aux as (
    select
        p.NU_INSCRICAO,
        p.NU.NOTA.REDACAO,
        p.TP.SEXO,
        q.Q005,
        row_number() over (partition by p.NU_INSCRICAO order by n.NU_NOTA_REDACAO desc) as rn
    from
        participantes p
    left join
        notas n on n.NU_INSCRICAO = p.NU_INSCRICAO
    left join
         questionario q on q.NU_INSCRICAO = p. NU_INSCRICAO
    where
        TP_ST_CONCLUSAO = 2
        and TP_COR_RACA = 4
        and TP_FAIXA_ETARIA = 2
)
select
    p.NU_INSCRICAO,
    p.NU.NOTA.REDACAO,
    p.TP.SEXO,
    avg(q.Q005) as media_pessoas_residencia
from
    aux
where
    rn = 1
group by 1, 2, 3
]
(Background on this error at: https://sqlalche.me/e/20/f405)